# Notebook Title
# Conformity Capacity Test for Amadeus API

---
### Purpose
The purpose of this notebook is to validate the behavior of the Amadeus API against the rate limits specified in its documentation.



According to the documentation, the following limits apply:
1. A maximum of 10 transactions per second, per user.
2. No more than 1 request every 100 ms.

This notebook implements a `ConformityCapacityTest` class to simulate API usage and verify if these rules are respected. 
The class is currently tailored to work with the Amadeus API due to the authentication and headers required, 
but it can be extended to other APIs by providing the necessary headers and configurations.

We will run a test with the provided rate limits and log any inconsistencies.


<img src="image-20250115-012858.png" width="" align="" />


### Observations
- The test results will include logs of each API request with timestamps and HTTP statuses.
- At the end of the test, a summary will show:
    - Total requests made
    - Number of 429 errors (Rate limit exceeded)
    - Number of connection errors
    - Any other errors observed

### Expected Behavior
If the Amadeus API adheres to its specified limits:
1. All requests should succeed if the interval between requests is at least 100 ms.
2. Any deviation (e.g., frequent 429 errors or unexplained connection errors) will indicate a discrepancy.

### Extensibility
The `ConformityCapacityTest` class can be easily extended for other APIs by:
- Providing the necessary headers for authentication. (WIP)
- Adjusting the rate limits as per the target API's documentation.

In [2]:
from Pricing4API.main.conformity_capacity import ConformityCapacityTest

RATE_WAIT_PERIOD = 0.1  
RATE_VALUE = 1  
TOTAL_REQUESTS = 10
URL = "https://test.api.amadeus.com/v1/reference-data/locations" 
PARAMS = {
    'keyword': 'LON', 
    'subType': 'AIRPORT'
}

# Note: Authentication is handled automatically within the ConformityCapacityTest class, and only works
# with the Amadeus API, not with other APIs. For demonstration purposes, we will provide the necessary headers.
# The headers can be adjusted but remains to be implemented.



The endpoint we are testing belongs to the Flighs API, which is not in the List of APIs that Amadeus qualifies as *AI and Partners' APIs* which have a different rate limit. It can be checked in the following link: [Amadeus Rate Limits](https://developers.amadeus.com/self-service/apis-docs/guides/developer-guides/api-rate-limits/).

In effect, the rate limits are set to 10 requests per second per user, and no more than 1 request every 100 ms. 

Let us see how it behaves:

In [ ]:
import nest_asyncio

nest_asyncio.apply()

test = ConformityCapacityTest(
    rate_wait_period=RATE_WAIT_PERIOD,
    total_requests=TOTAL_REQUESTS,
    rate_value=RATE_VALUE,
    url=URL,
    params=PARAMS
)


test.execute()


## Analysis of Results

This code demonstrates that the Amadeus API appears to be providing misleading information regarding its rate limits. 

According to the official documentation:
1. **Maximum of 10 transactions per second, per user**.
2. **No more than 1 request every 100ms**.

However, during the execution of this test:
- **429 (Rate Limit Exceeded)** errors were observed, even when the script adhered to the specified 100ms interval between requests.
- This inconsistency raises concerns about whether the actual implementation of rate limiting by Amadeus matches its published specifications.

### Key Findings:
1. The script respects the rate limit by ensuring only 1 request is made every 100ms, yet multiple 429 errors are encountered.
2. This behavior suggests that the actual rate-limiting mechanism of Amadeus may impose stricter limits than documented or that other factors (e.g., internal system latencies) are at play.


### Conclusion:
This test demonstrates the importance of verifying API documentation with real-world experiments, as discrepancies like these can significantly impact system design and reliability.


---

## Extended Test: Increasing the Interval to 500ms

In this section, we tested the Amadeus API by increasing the interval between requests to **500ms**  (2 requests per second). This test aims to observe if a larger interval reduces the number of `429 (Rate Limit Exceeded)` errors.

### Observations:
1. **Reduction in Errors**: Compared to the 100ms interval, the number of `429` errors decreased significantly when using a 500ms interval.
2. **Inconsistent Behavior**: Despite spacing the requests at 500ms intervals, we still encountered some `429` errors, which raises questions about the exact implementation of Amadeus's rate-limiting mechanism.
3. **Cannot Guarantee Reliability**: The continued presence of occasional errors, even with conservative spacing, prevents us from confidently relying on the documented limits.

### Key Findings:
- Increasing the request interval appears to improve reliability, but it does not eliminate errors entirely.
- This inconsistency indicates that the Amadeus rate-limiting rules may involve additional factors (e.g., user-specific quotas, shared resources, or undocumented constraints).

### Try it Out:

You can test this script by tweaking the configuration parameters `RATE_WAIT_PERIOD` and `RATE_VALUE`:

- **`RATE_WAIT_PERIOD`**: The span of time between individual requests (in seconds). For instance, `0.1` represents a 100ms interval.
- **`RATE_VALUE`**: The number of requests sent as a "burst" within each interval. A value of `1` ensures that only one request is sent every `RATE_WAIT_PERIOD`.

**Example**:
```python
RATE_WAIT_PERIOD = 0.1  # Time interval between requests (100ms)
RATE_VALUE = 1  # Single request per interval
TOTAL_REQUESTS = 15  # Total number of requests to simulate
```

---

### ⚠️ Disclaimer: Be Careful with the Quota

In the **Test Plan** (free tier), you are limited to **2,000 requests per month**. Ensure that you monitor your usage to avoid exceeding this quota.

If you exceed the limit, you may encounter errors or temporary access restrictions.

To avoid depleting your quota:
- Use conservative values for `TOTAL_REQUESTS`.
- Experiment with endpoints and parameters to focus only on relevant queries.

#### Alternative Endpoint Example:
```python
URL = "https://test.api.amadeus.com/v1/reference-data/locations"
PARAMS = {
    "keyword": "LON",
    "subType": "AIRPORT"
}



In [ ]:
RATE_WAIT_PERIOD = 0.5 
RATE_VALUE = 1  
TOTAL_REQUESTS = 50

test = ConformityCapacityTest(
    rate_wait_period=RATE_WAIT_PERIOD,
    total_requests=TOTAL_REQUESTS,
    rate_value=RATE_VALUE,
    url=URL,
    params=PARAMS
)


test.execute()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bfc07943-bcc8-48e2-8be3-26469867b14f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>